In [2]:
import os
from IPython.display import Image
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

In [3]:
def order_atoms(filename='atom_list.txt'):
    """list out the txt file containing all atoms in the molecule, order the same as JANPA output and eliminate whitespace"""
    with open(filename) as f:
        atoms = f.readlines()
        atoms = [atom.strip() for atom in atoms]
    
    return atoms

In [4]:
atoms = order_atoms()

In [5]:
def add_atoms(atoms):
    """add atoms and correct bonds to build molecule"""
    # create molecule under RWMol class, easily editable
    mol = Chem.RWMol()
    # call function to order atoms
    order_atoms()
    # add all atoms to molecule
    for atom in atoms:
        mol.AddAtom(Chem.Atom(atom))
    return mol
        

In [6]:
output = add_atoms(atoms)
type(output)

rdkit.Chem.rdchem.RWMol

In [25]:
def build_molecule(bonding_matrix):
    """Takes connection matrix and builds the molecule with RDKit"""
    # add bonds from connection matrix built from bondorders.py
    bonds = bonding_matrix
    # ensure matrix is square
    assert bonding_matrix.shape[0] == bonding_matrix.shape[1], 'matrix must be square!'
    # ensure diagonals don't equal 0
    for i in range(bonding_matrix.shape[0]):
        assert bonding_matrix[i,i] != 0, 'diagonals cannot equal 0'
    for i, j in zip(*np.triu_indices_from(bonds, 1)):
        if np.isclose(bonds[i, j], 1):
            mol.AddBond(int(i), int(j), Chem.BondType.SINGLE)
        elif np.isclose(bonds[i, j], 1.5):
            mol.AddBond(int(i), int(j), Chem.BondType.AROMATIC)
        elif np.isclose(bonds[i, j], 2):
            mol.AddBond(int(i), int(j), Chem.BondType.DOUBLE)
        elif np.isclose(bonds[i, j], 3):
            mol.AddBond(int(i), int(j), Chem.BondType.TRIPLE)
        elif np.isclose(bonds[i, j], 0):
            pass
        else:
            raise Exception('indeterminate or nonphysical bond type!')
    # Do not show hydrogens
    mol = add_atoms(atoms)
    for at in mol.GetAtoms():
        at.SetNoImplicit(True)
    return mol

In [8]:
def draw_molecule(mol):
    """represent built molecule as 2 and 3 dimensional images"""
    # use RDKit internal check to verify molecule is physical
    Chem.SanitizeMol(mol)
    mol_noH = Chem.RemoveHs(mol)
    # 2d representation of molecule
    two_d = Draw.MolToImage(mol_noH)
    # 3d picture
    three_d = Image('mol.png', width=400)
    inchi = "InChI: {}".format(Chem.MolToInchi(mol_noH))
    smiles = "SMILES: {}".format(Chem.MolToSmiles(mol_noH))
    
    return two_d, three_d, inchi, smiles

## Temporary space for testing

In [16]:
def test_order_atoms():
    """   """
    atoms = order_atoms('test_atom_list.txt')
    assert sum(line.isspace() for line in atoms) == 0, 'order_atoms function failed to eliminate whitespace'
    actual_length = len(atoms)
    assert len(atoms) == 43, 'molecule should have 47 atoms,' + str(actual_length) + ' was counted'
    return True


def test_add_atoms():
    result = add_atoms(atoms)
    assert type(result) == Chem.rdchem.RWMol
    return True

def test_build_molecule():
        bond_matrix1 = np.array([[2,0,'a'], [1,1,0],[0,2,1]])
        bond_matrix2 = np.array([[1,1,0],[0,2,1]])
        bond_matrix3 = np.array([[1,0,1.2],[0,2,0],[0,0,1]])
        bond_matrix4 = np.array([[1,0,0],[0,1,0],[0,2,0]])
        atoms = order_atoms('test_atom_list.txt')
        mol = add_atoms(atoms)
        # try string value
        try:
            build_molecule(bond_matrix1)
        except Exception as err:
            print(err)
        else:
            raise Exception('did not catch string input in bonding matrix')
        # try non-square matrix
        try:
            build_molecule(bond_matrix2)
        except Exception as err:
            print(err)
        else:
            raise Exception('did not catch non-square matrix')
        # try float value
        try:
            build_molecule(bond_matrix3)
        except Exception as err:
            print(err)
        else:
            raise Exception('did not catch float value not close to an integer')
        # try 0 value diagonal
        try:
            build_molecule(bond_matrix4)
        except Exception as err:
            print(err)
        else:
            for i in range(bonding_matrix.shape[0]):
                print(bonding_matrix[i,i])
        #raise Exception('did not catch diagonal value of 0')
            
        return True
        


In [17]:
test_build_molecule()

ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
matrix must be square!
indeterminate or nonphysical bond type!
diagonals cannot equal 0


True

In [26]:
bond_matrix4 = np.array([[1,0,0],[0,2,0],[0,2,3]])

try:
    build_molecule(bond_matrix4)
except Exception as err:
    print(err)
else:
    raise Exception('aweofij')
#for i in range(bonding_matrix.shape[0]):
#print(bonding_matrix[i,i])

Exception: aweofij

In [27]:
bond_matrix4 = np.array([[1,0,0],[0,2,0],[0,2,3]])
build_molecule(bond_matrix4)